Prompting OpenAI models

In [ ]:
! pip install langchain-huggingface

In [ ]:
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
  model_id="crumb/nano-mistral",
  task="text-generation",
  pipeline_kwargs={"max_new_tokens": 100},
)

prompt = "What is LangChain?"
llm.invoke(prompt)

------

Prompt Templates
- Receipes for defining prompts for LLMS

In [ ]:
from langchain_core.prompts import PromptTemplate

template = "Explain this concept simply and concisely: {concept}"
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.invoke({"concept": "Prompting LLMs"})
print(prompt)

In [ ]:
# | (Pipe) is used to chain components together
llm_chain = prompt_template | llm

concept = "Prompting LLMs"
print(llm_chain.invoke({"concept": concept}))

Chat roles
- System: Defines the behavior of the assistant
- Human: User input
- AI: Model response

In [ ]:
# Creating chat models with roles
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a calculator that responds with math."),
    ("user", "Answer this math question What is two plus two?"),
    ("ai", "2+2 = 4"),
    ("human", "Answer this math question: {math}"),
  ]
)

llm_chain = template | llm
math = "What is five times five?"
response = llm_chain.invoke({"math": math})
print(response)

-----

Few-shot prompting

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate.from_template("Question: {question}\nAnswer: {answer}")
prompt = example_prompt.invoke(
    {
        "question": "What is the capital of Italy?",
        "answer": "Rome",
    }
)

print(prompt.text)

In [ ]:
examples = [{
    "question": "Does Henry Campbell have any pets?",
    "answer" : "Henry Campbell has a dog called Pluto."
}]

prompt_template = FewShotPromptTemplate(
  examples = examples, # List of Dictionaries with "question" and "answer" keys
  example_prompt=example_prompt, # Formatted template
  suffix="Question: {input}", # Suffix to add to the input
  input_variables=["input"], # Input variables to use in the template
)

prompt = prompt_template.invoke({"input": "What is the namne of Henry Campbell's dog?"})
print(prompt.text)

In [ ]:
llm_chain = prompt_template | llm
response = llm_chain.invoke({"input": "What is the name of Henry Campbell's dog?"})
print(response)

------

Sequential Chains

In [ ]:
from langchain_core.output_parsers import StrOutputParser

destination_prompt = PromptTemplate(
  input_variables=["destination"],
  template="I am planning a trip to {destination}. Can you suggest some activities to do there?"
)

activities_prompt = PromptTemplate(
  input_variables=["activities"],
  template="I have only one day, so can you create itinerary from your top three activities: {activities}"
)


seq_chain = ({"activities" : destination_prompt | llm | StrOutputParser() }) | activities_prompt | llm | StrOutputParser()

-------

Agents
- Agents : Use LLMs to take actions
- Tools: Functions called by agents

In [ ]:
! pip install langgraph==0.2.74

In [ ]:
# ReAct Agent : Reason + Act

from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits.load_tools import load_tools

tools = load_tools(['llm-math'], llm=llm)
agent = create_react_agent(llm, tools)

messages = agent.invoke({"messages":[("human", "What is the square root of 101?")]})
print(messages['messages'][-1].content)

In [ ]:
print(tools[0].name)
print(tools[0].description)
print(tools[0].return_direct) # whether the agent should stop after the tool call or continue reasoning
print(tools[0].args) # the arguments that the tool expects

----

Define a custom function

In [ ]:
from langchain_core.tools import tool

@tool
def financial_report(company_name: str, revenue: int, expense: int) -> str:
  """ Generate a financial report for a company that calculates net income."""
  net_income = revenue - expense
  
  report = f"Financial Report for {company_name}:\n"
  report += f"Revenue: ${revenue}\n"
  report += f"Expenses: ${expense}\n"
  report += f"Net Income: ${net_income}\n"
  return report

print(financial_report("LemondateStand", 100, 50))